In [111]:
# Import hdf reader, declare directory and files
import h5py
import numpy as np
import pandas
import time
dir='/SPENCEdata/Research/database/DMSP/20071227/'
file1='dms_20071227_16e.001__Baddeley_et_al_2007.hdf5'
file2='dms_20071227_16s1.001__Baddeley_et_al_2007.hdf5'
file3='dms_20071227_16s4.001__Baddeley_et_al_2007.hdf5'

In [14]:
# Open file
f1 = h5py.File(dir+file1, 'r')

In [16]:
print("Keys: %s" % f1.keys())

Keys: [u'Data', u'Metadata']


In [27]:
group_keys = f1.keys()
print(group_keys)

[u'Data', u'Metadata']


In [29]:
data1 = f1['Data']
mData1 = f1['Metadata']

In [31]:
data1.shape

u'/Data'

In [55]:
for name in data1:
    print name
table1 = data1.get('Table Layout')
array1 = data1.get('Array Layout')

Array Layout
Table Layout


In [54]:
# Table1 properties
print(table1.id)
print(table1.dims)
print(table1.file)
print(table1.name)
print(table1.attrs)

<Dimensions of HDF5 object at 139754698409512>
<HDF5 file "dms_20071227_16e.001__Baddeley_et_al_2007.hdf5" (mode r)>
/Data/Table Layout
<Attributes of HDF5 object at 139754698409512>


In [59]:
# array1 properties
for name in array1:
    print name
parm1ds = array1.get('1D Parameters')
parm2ds = array1.get('2D Parameters')
layoutDescr = array1.get('Layout Description')
ch_energy =  array1.get('ch_energy')
timestamps =  array1.get('timestamps')

1D Parameters
2D Parameters
Layout Description
ch_energy
timestamps


In [91]:
# show members of parm1ds
k=0
for name in parm1ds:
    print(k,name,parm1ds.get(name).shape,parm1ds.get(name).dtype)
    # print(name)
    k += 1

(0, u'Data Parameters', (13,), dtype([('mnemonic', 'S10'), ('description', 'S39'), ('isError', '<i8'), ('units', 'S7'), ('category', 'S36')]))
(1, u'el_i_ener', (86400,), dtype('float64'))
(2, u'el_i_flux', (86400,), dtype('float64'))
(3, u'el_m_ener', (86400,), dtype('float64'))
(4, u'gdalt', (86400,), dtype('float64'))
(5, u'gdlat', (86400,), dtype('float64'))
(6, u'glon', (86400,), dtype('float64'))
(7, u'ion_i_ener', (86400,), dtype('float64'))
(8, u'ion_i_flux', (86400,), dtype('float64'))
(9, u'ion_m_ener', (86400,), dtype('float64'))
(10, u'mlat', (86400,), dtype('float64'))
(11, u'mlong', (86400,), dtype('float64'))
(12, u'mlt', (86400,), dtype('float64'))
(13, u'sat_id', (86400,), dtype('int64'))


In [79]:
parm1dsdtype=[('el_i_ener',np.float64),('el_i_flux',np.float64),('el_m_ener',np.float64),('gdalt',np.float64),('gdlat',np.float64),('glon',np.float64),('ion_i_ener',np.float64),('ion_i_flux',np.float64),('ion_m_ener',np.float64),('mlat',np.float64),('mlong',np.float64),('mlt',np.float64),('sat_id',np.float64)]

In [93]:
dmsp = np.recarray((86400,),dtype=parm1dsdtype)
dmsp.el_i_ener = parm1ds.get('el_i_ener')
dmsp.el_i_flux = parm1ds.get('el_i_flux')
dmsp.el_m_ener = parm1ds.get('el_m_ener')
dmsp.gdalt = parm1ds.get('gdalt')
dmsp.gdlat = parm1ds.get('gdlat')
dmsp.glon = parm1ds.get('glon')
dmsp.ion_i_ener = parm1ds.get('ion_i_ener')
dmsp.ion_i_flux = parm1ds.get('ion_i_flux')
dmsp.ion_m_ener = parm1ds.get('ion_m_ener')
dmsp.mlat = parm1ds.get('mlat')
dmsp.mlong = parm1ds.get('mlong')
dmsp.mlt = parm1ds.get('mlt')
dmsp.sat_id = parm1ds.get('sat_id')

In [109]:
# Now define t2s function
def t2s(x): return time.strftime("%Y-%m-%d/%H:%M:%S", time.gmtime(x))

In [112]:
# Can we convert timestamps to strings?
for i in range(0,2):
    # print(time.strftime("%Y-%m-%d/%H:%M:%S", time.gmtime(timestamps[i])))
    print(t2s(timestamps[i]))

2007-12-27/00:00:00
2007-12-27/00:00:01


In [121]:
# Try mapping
tStampStr = list(map(t2s,timestamps))

First : 2007-12-27/00:00:00
Last  : 2007-12-27/00:00:00


In [122]:
print("First : " + tStampStr[0])
print("Last  : " + tStampStr[-1])

First : 2007-12-27/00:00:00
Last  : 2007-12-27/23:59:58


In [138]:
# Hmm, those seem all right. So where is the diff greater than one second?
tDiff = timestamps[1:-1]-timestamps[0:-2]
gt1Ind = ((tDiff > 1).ravel().nonzero())[0]

In [142]:
gt1Ind

array([21848, 43196, 46724, 46742, 46760, 46763, 46777, 46780, 46790,
       46793, 46796, 46804, 46805, 46808, 46811, 48546, 74794, 82225])

In [146]:
# Now see the diff
for k in range(0,gt1Ind.size-1):
    print(tDiff[gt1Ind[k]],tStampStr[gt1Ind[k]],tStampStr[gt1Ind[k]+1])

(2.0, '2007-12-27/06:04:08', '2007-12-27/06:04:10')
(3.0, '2007-12-27/11:59:57', '2007-12-27/12:00:00')
(2.0, '2007-12-27/12:58:47', '2007-12-27/12:58:49')
(2.0, '2007-12-27/12:59:06', '2007-12-27/12:59:08')
(2.0, '2007-12-27/12:59:25', '2007-12-27/12:59:27')
(2.0, '2007-12-27/12:59:29', '2007-12-27/12:59:31')
(2.0, '2007-12-27/12:59:44', '2007-12-27/12:59:46')
(2.0, '2007-12-27/12:59:48', '2007-12-27/12:59:50')
(2.0, '2007-12-27/12:59:59', '2007-12-27/13:00:01')
(2.0, '2007-12-27/13:00:03', '2007-12-27/13:00:05')
(2.0, '2007-12-27/13:00:07', '2007-12-27/13:00:09')
(2.0, '2007-12-27/13:00:16', '2007-12-27/13:00:18')
(2.0, '2007-12-27/13:00:18', '2007-12-27/13:00:20')
(2.0, '2007-12-27/13:00:22', '2007-12-27/13:00:24')
(2.0, '2007-12-27/13:00:26', '2007-12-27/13:00:28')
(2.0, '2007-12-27/13:29:22', '2007-12-27/13:29:24')
(2.0, '2007-12-27/20:46:51', '2007-12-27/20:46:53')
